In [2]:
import numpy as np
import nltk
import random
import csv
from sklearn import linear_model

In [4]:
with open('/home/xuhui/10002/Machine_Learning/R-net-master/glove.840B.300d.txt') as f:
    word2vec = {}
    for index, line in enumerate(f):
        line =line.strip().split(' ')
        word2vec[line[0]] = np.array(line[1:],dtype="float")

In [5]:
def get_question():
    q_dict ={
        'A1':'What is the objective / aim of this paper ?',
        'A2':'What problem(s) does this paper address ?',
        'A41':'What method / approach does this paper propose ?',
        'A51':'What is this method based on ?',
        'A61':'How does the proposed method differ from previous methods / approaches ?',
        'A42':'What model does this paper propose ?',
        'A52':'What is this model based on ?',
        'A62':'How does the proposed model differ from previous models ?',
        'A43':'What algorithm does this paper propose ?',
        'A53':'What is this algorithm based on ?',
        'A63':'How does the proposed algorithm differ from previous algorithms ?',
        'A44':'What framework does this paper propose ?',
        'A54':'What is this framework based on ?',
        'A64':'How does the proposed framework differ from previous frameworks ?',
        'A45':'What datasetdoes this paper propose ?',
        'A7':'What experiment does this paper carry out to evaluate the result ?',
        'A81':'What does the result of this paper show(demonstrated by the experiment)',
        'A82':'What does the result of this paper show(demonstrated by the experiment)',
        'A83':'What does the result of this paper show(demonstrated by the experiment)',
        'A10':'How does this result outperform existing work ?' 
        }
    return q_dict

In [6]:
def get_liter_questions(ques_ID):
    return get_question()[ques_ID]

In [7]:
def cal_sen_vec(content):
    content2vec = np.zeros(300)
    for i in content:
        content2vec += word2vec.get(i,np.zeros(300))
    return content2vec

In [8]:
train_data = []
train_label = []
csvFile1 = open('/home/xuhui/10002/Machine_Learning/train.csv', 'r', encoding='utf-8')
train_file = csv.reader(csvFile1)
for line in train_file:
    # 忽略第一行
    if train_file.line_num == 1:
        continue
        
    ques_ID = line[0]    
    QA_abs = line[1]
    answer_text = line[2]
    
    question = get_liter_questions(ques_ID)
    
    ques_tokens = nltk.word_tokenize(question)
    abs_tokens = nltk.word_tokenize(QA_abs)
    ans_tokens = nltk.word_tokenize(answer_text)
    
    ques_vec = cal_sen_vec(ques_tokens)
    abs_vec = cal_sen_vec(abs_tokens)
    ans_vec = cal_sen_vec(ans_tokens)
    
    train_data.append(np.concatenate((ques_vec,abs_vec,ans_vec)))
    train_label.append(1)
    
    
    start, end = sorted(random.sample(range(len(QA_abs)),2))
    false_ans = QA_abs[start:end]
    false_tokens = nltk.word_tokenize(false_ans)
    false_vec = cal_sen_vec(false_tokens)
    
    train_data.append(np.concatenate((ques_vec,abs_vec,false_vec)))
    train_label.append(0)

train_data = np.array(train_data)
train_label = np.array(train_label)

In [10]:
test_data = []
test_label = []
csvFile1 = open('/home/xuhui/10002/Machine_Learning/test.csv', 'r', encoding='utf-8')
test_file = csv.reader(csvFile1)

csvFile2 = open('/home/xuhui/10002/Machine_Learning/sample.csv', 'r', encoding='utf-8')
sample_file = csv.reader(csvFile2)
ID2Category = {}
for line in sample_file:
    if sample_file.line_num == 1:
        continue
    ID2Category[line[0]] = line[1]

for line in test_file:
    # 忽略第一行
    if test_file.line_num == 1:
        continue
    
    ques_ID = line[1]    
    QA_abs = line[2]
    if ID2Category[line[0]]=='1':
        answer_text = line[3]
        false_answer_text = line[4]
    elif ID2Category[line[0]]=='2':
        answer_text = line[4]
        false_answer_text = line[3]
    question = get_liter_questions(ques_ID)
    
    ques_tokens = nltk.word_tokenize(question)
    abs_tokens = nltk.word_tokenize(QA_abs)
    ans_tokens = nltk.word_tokenize(answer_text)
    
    ques_vec = cal_sen_vec(ques_tokens)
    abs_vec = cal_sen_vec(abs_tokens)
    ans_vec = cal_sen_vec(ans_tokens)
    
    test_data.append(np.concatenate((ques_vec,abs_vec,ans_vec)))
    test_label.append(1)
    
    false_ans_tokens = nltk.word_tokenize(false_answer_text)
    false_ans_vec = cal_sen_vec(false_ans_tokens)
    test_data.append(np.concatenate((ques_vec,abs_vec,false_ans_vec)))
    test_label.append(0)
test_data = np.array(test_data)
test_label = np.array(test_label)

In [11]:
LR=linear_model.LogisticRegression(penalty='l2',tol=1e-6,multi_class='multinomial',solver='newton-cg')
LR.fit(train_data, train_label)

/home/xuhui/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='newton-cg',
          tol=1e-06, verbose=0, warm_start=False)

In [26]:
pre = LR.predict(test_data)

In [34]:
precision = ((test_label==1)*(pre==1)).sum()/(pre==1).sum()
recall = ((test_label==1)*(pre==1)).sum()/(test_label==1).sum()
F1 = precision*recall/(precision+recall)
print('Precision: %f, F1-score: %f'%(precision,F1))

Precision: 0.565481, F1-score: 0.348324
